* 08.02.23
* Yaromir Kapustin
* ver0.2

# <center>ДЗ 25. Прогноз просрочки кредита заемщиком</center>

### Решить задачу просрочки кредита используя для прогнозирования нейронные сети. Посмотреть работу с несбалансированными данными. 

In [130]:
import numpy as np
import pandas as pd
import seaborn as sns
from matplotlib import pyplot as plt

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

import tensorflow as tf
from tensorflow import keras

import warnings
warnings.filterwarnings('ignore')

### Описание набора данных

Данные представляют собой исторические данные по 251503 заемщикам (https://www.kaggle.com/c/GiveMeSomeCredit/data ). Задача является бинарной классификацией. Цель - предсказать будет ли тот или иной заемщик испытывать финансовые трудности в ближайшие 2 года, т.е. будет ли просрочка по займу. Выборка разделена на тренировочную и тестовую ( 150000 в тренировочной части, 101503 в тестовой).

Данная решаемая задача будет ценна как кредиторам, так и заемщикам для оценки способности вернуть долг вовремя.

### Описание признаков

* 1 - __SeriousDlqin2yrs__ - будет ли просрочка более 90 дней в ближайшие 2 года (целевая метка) (Да/Нет)
* 2 - __RevolvingUtilizationOfUnsecuredLines__ - общий баланс по кредитным картам и кредитным линиям, за исключением задолженностей по недвижимости задолженности по взносам, деленные на сумму кредитных лимитов (проценты)
* 3 - __age__ - Возраст заемщика (в годах)
* 4 - __NumberOfTime30-59DaysPastDueNotWorse__ - количество просрочек в 30-59 дней (Целое)
* 5 - __DebtRatio__ - коэффициент задолженности, т.е. сумма ежемесячных платежей по долгам, алиментов и расходов на проживание, деленная на месячный доналоговый доход (проценты)
* 6 - __Monthly Income__ - месячный доход (число с плавающей точкой)
* 7 - __NumberOfOpenCreditLinesAndLoans__ - количество открытых кредитов и кредитных линий (Целое)
* 8 - __NumberOfTimes90DaysLate__ - количество просрочек более 90 дней (Целое)
* 9 - __NumberRealEstateLoansOrLines__ - количество ипотечных кредитов и кредитов на недвижимость, включая кредитные линии домашнего капитала (Целое)
* 10 - __NumberOfTime60-89DaysPastDueNotWorse__ - количество просрочек в 60-89 дней за последние 2 года (Целое)
* 11 - __NumberOfDependents__ - количество иждивенцев в семье (исключая самих заемщиков) (Целое)

In [131]:
df = pd.read_csv('cs-training.csv', index_col=0)

In [132]:
df.head()

,SeriousDlqin2yrs,RevolvingUtilizationOfUnsecuredLines,age,NumberOfTime30-59DaysPastDueNotWorse,DebtRatio,MonthlyIncome,NumberOfOpenCreditLinesAndLoans,NumberOfTimes90DaysLate,NumberRealEstateLoansOrLines,NumberOfTime60-89DaysPastDueNotWorse,NumberOfDependents
1,1,0.766127,45,2,0.802982,9120.0,13,0,6,0,2.0
2,0,0.957151,40,0,0.121876,2600.0,4,0,0,0,1.0
3,0,0.658180,38,1,0.085113,3042.0,2,1,0,0,0.0
4,0,0.233810,30,0,0.036050,3300.0,5,0,0,0,0.0
5,0,0.907239,49,1,0.024926,63588.0,7,0,1,0,0.0


In [133]:
print(df.shape)

(150000, 11)


Проверьте наличие категориальных признаков, а также соответствие содержимого данных их типам

In [134]:
#все столбцы числовые
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 150000 entries, 1 to 150000
Data columns (total 11 columns):
 #   Column                                Non-Null Count   Dtype  
---  ------                                --------------   -----  
 0   SeriousDlqin2yrs                      150000 non-null  int64  
 1   RevolvingUtilizationOfUnsecuredLines  150000 non-null  float64
 2   age                                   150000 non-null  int64  
 3   NumberOfTime30-59DaysPastDueNotWorse  150000 non-null  int64  
 4   DebtRatio                             150000 non-null  float64
 5   MonthlyIncome                         120269 non-null  float64
 6   NumberOfOpenCreditLinesAndLoans       150000 non-null  int64  
 7   NumberOfTimes90DaysLate               150000 non-null  int64  
 8   NumberRealEstateLoansOrLines          150000 non-null  int64  
 9   NumberOfTime60-89DaysPastDueNotWorse  150000 non-null  int64  
 10  NumberOfDependents                    146076 non-null  float64
dtype

Проверьте наличие NaN, при необходимости осуществите замену

О чем свидетельствуют пропуски в данных, чем их лучше заменить? Осуществите необходимую замену...

In [135]:
# колонки MonthlyIncome и NumberOfDependents содержат пропуски
df.MonthlyIncome = df.MonthlyIncome.fillna(df.MonthlyIncome.median())
df.NumberOfDependents = df.NumberOfDependents.fillna(df.NumberOfDependents.median())

In [136]:
#удалю выбросы
df = df.loc[df['age'] < 92]
df = df.loc[df['NumberOfDependents'] < 10]
df = df.loc[df['NumberOfOpenCreditLinesAndLoans'] < 36]
df = df.loc[df['NumberRealEstateLoansOrLines'] < 12]
df = df.loc[df['NumberOfTimes90DaysLate'] < 7]
df = df.loc[df['NumberOfTime30-59DaysPastDueNotWorse'] < 8]
df = df.loc[df['NumberOfTime60-89DaysPastDueNotWorse'] < 5]
df.DebtRatio = df.DebtRatio.round(decimals = 2)
df = df.loc[df['DebtRatio'] != 0]
df.MonthlyIncome = df.MonthlyIncome.round(decimals = -3)
df.RevolvingUtilizationOfUnsecuredLines = df.RevolvingUtilizationOfUnsecuredLines.round(decimals = 1)
df = df.loc[df['RevolvingUtilizationOfUnsecuredLines'] < 6]

In [137]:
#результат чистки
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 141132 entries, 1 to 150000
Data columns (total 11 columns):
 #   Column                                Non-Null Count   Dtype  
---  ------                                --------------   -----  
 0   SeriousDlqin2yrs                      141132 non-null  int64  
 1   RevolvingUtilizationOfUnsecuredLines  141132 non-null  float64
 2   age                                   141132 non-null  int64  
 3   NumberOfTime30-59DaysPastDueNotWorse  141132 non-null  int64  
 4   DebtRatio                             141132 non-null  float64
 5   MonthlyIncome                         141132 non-null  float64
 6   NumberOfOpenCreditLinesAndLoans       141132 non-null  int64  
 7   NumberOfTimes90DaysLate               141132 non-null  int64  
 8   NumberRealEstateLoansOrLines          141132 non-null  int64  
 9   NumberOfTime60-89DaysPastDueNotWorse  141132 non-null  int64  
 10  NumberOfDependents                    141132 non-null  float64
dtype

In [138]:
scaler = StandardScaler()

In [139]:
y = df["SeriousDlqin2yrs"]
X = df.drop(["SeriousDlqin2yrs"], axis=1)

X = scaler.fit_transform(X)

In [140]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)

In [141]:
model = tf.keras.models.Sequential([
    tf.keras.layers.Flatten(input_shape=X_train.shape[1:]),
    tf.keras.layers.Dense(30, activation="relu"), 
    tf.keras.layers.Dense(10, activation="relu"),
    tf.keras.layers.Dense(1, activation="sigmoid")
])

In [142]:
model.summary()

Model: "sequential_9"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 flatten_5 (Flatten)         (None, 10)                0         
                                                                 
 dense_32 (Dense)            (None, 30)                330       
                                                                 
 dense_33 (Dense)            (None, 10)                310       
                                                                 
 dense_34 (Dense)            (None, 1)                 11        
                                                                 
Total params: 651
Trainable params: 651
Non-trainable params: 0
_________________________________________________________________


In [143]:
model.compile(loss=keras.losses.BinaryCrossentropy(),
              optimizer=keras.optimizers.Adam(learning_rate=0.001), 
              metrics=keras.metrics.AUC())

In [144]:
history = model.fit(X_train, y_train, epochs=50, validation_data=(X_test, y_test), batch_size=256)

Epoch 1/50
414/414 [==============================] - 1s 2ms/step - loss: 0.2293 - auc_4: 0.7557 - val_loss: 0.1875 - val_auc_4: 0.8350
Epoch 2/50
414/414 [==============================] - 1s 2ms/step - loss: 0.1861 - auc_4: 0.8419 - val_loss: 0.1819 - val_auc_4: 0.8507
Epoch 3/50
414/414 [==============================] - 1s 2ms/step - loss: 0.1832 - auc_4: 0.8492 - val_loss: 0.1797 - val_auc_4: 0.8557
Epoch 4/50
414/414 [==============================] - 1s 2ms/step - loss: 0.1820 - auc_4: 0.8526 - val_loss: 0.1790 - val_auc_4: 0.8573
Epoch 5/50
414/414 [==============================] - 1s 1ms/step - loss: 0.1812 - auc_4: 0.8544 - val_loss: 0.1787 - val_auc_4: 0.8578
Epoch 6/50
414/414 [==============================] - 1s 2ms/step - loss: 0.1809 - auc_4: 0.8554 - val_loss: 0.1784 - val_auc_4: 0.8591
Epoch 7/50
414/414 [==============================] - 1s 2ms/step - loss: 0.1805 - auc_4: 0.8560 - val_loss: 0.1783 - val_auc_4: 0.8591
Epoch 8/50
414/414 [============================

In [145]:
scores = model.evaluate(X_test, y_test, verbose=0)

print("AUC: " + str(scores[1]*100))

AUC: 86.12301349639893
